In [ ]:
!pip install geemap

In [5]:
import geemap

In [16]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [17]:
import ee
import geemap

# Initialize the Earth Engine API
ee.Initialize()

# Define the area of interest (latitude, longitude)
point = ee.Geometry.Polygon([
        [[-8.570156845611686, 52.00904254772663],
         [-8.570156845611686, 51.7292195887807],
         [-7.959042343658562, 51.7292195887807],
         [-7.959042343658562, 52.00904254772663]]
    ])

# Initialize the map
Map = geemap.Map(center=[51.8691, -8.2646], zoom=10)  # Adjust center and zoom as needed

# Add the area of interest to the map
Map.addLayer(point, {'color': 'red'}, 'Area of Interest')

# Display the map
Map


Map(center=[51.8691, -8.2646], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

In [18]:
import ee
import datetime

# Initialize the Earth Engine API
ee.Initialize()

# Define the area of interest (latitude, longitude, and buffer)
point = ee.Geometry.Polygon([
        [[-8.570156845611686, 52.00904254772663],
         [-8.570156845611686, 51.7292195887807],
         [-7.959042343658562, 51.7292195887807],
         [-7.959042343658562, 52.00904254772663]]
    ])

area_of_interest = point.buffer(5000)  # 5km buffer

# Define time range (start and end date)
start_date = '2015-01-01'
end_date = '2023-12-31'

# Fetch Landsat 8 images for the defined area and time range
landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                     .filterBounds(area_of_interest) \
                     .filterDate(start_date, end_date) \
                     .filterMetadata('CLOUD_COVER', 'less_than', 20)

# Get the first image from the collection
first_image = landsat_collection.first()

# Print image information
print(first_image.getInfo())

# Export the image to Google Drive (you'll need to have Google Drive connected)
task = ee.batch.Export.image.toDrive(
    image=first_image,
    description='Landsat_image',
    folder='EarthEngineImages',
    scale=30,
    region=area_of_interest
)
task.start()

print("Image export task completed...")


{'type': 'Image', 'bands': [{'id': 'SR_B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7811, 7911], 'crs': 'EPSG:32629', 'crs_transform': [30, 0, 501585, 0, -30, 5846415]}, {'id': 'SR_B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7811, 7911], 'crs': 'EPSG:32629', 'crs_transform': [30, 0, 501585, 0, -30, 5846415]}, {'id': 'SR_B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7811, 7911], 'crs': 'EPSG:32629', 'crs_transform': [30, 0, 501585, 0, -30, 5846415]}, {'id': 'SR_B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7811, 7911], 'crs': 'EPSG:32629', 'crs_transform': [30, 0, 501585, 0, -30, 5846415]}, {'id': 'SR_B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7811, 7911], 'crs': 'EPSG:32629', 'crs_transform': [30, 0, 501585, 0, -30